# Интеллектуальный анализ данных – весна 2025
# Домашнее задание 6: классификация текстов

Правила:



*   Домашнее задание оценивается в 10 баллов.
*   Можно использовать без доказательства любые результаты, встречавшиеся на лекциях или семинарах по курсу, если получение этих результатов не является вопросом задания.
*  Можно использовать любые свободные источники с *обязательным* указанием ссылки на них.
*  Плагиат не допускается. При обнаружении случаев списывания, 0 за работу выставляется всем участникам нарушения, даже если можно установить, кто у кого списал.
*  Старайтесь сделать код как можно более оптимальным. В частности, будет штрафоваться использование циклов в тех случаях, когда операцию можно совершить при помощи инструментов библиотек, о которых рассказывалось в курсе.
* Если в задании есть вопрос на рассуждение, то за отсутствие ответа на него балл за задание будет снижен вполовину.

В этом домашнем задании вам предстоит построить классификатор текстов.

Будем предсказывать эмоциональную окраску твиттов о коронавирусе.



In [1]:
import numpy as np
import pandas as pd
from typing import List
import matplotlib.pyplot as plt
import seaborn as sns
from string import punctuation

In [2]:
df = pd.read_csv('tweets_coronavirus.csv', encoding='latin-1')
df.sample(4)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
6407,11581,56533,NaN,19-03-2020,Our communities are facing the most challengin...,Negative
4214,8921,53873,Russia,18-03-2020,"If you think #COVID2019, the Wall-Street under...",Extremely Negative
28513,38782,83734,"Covington, LA",08-04-2020,A few bright spots are emerging amid the chall...,Positive
22951,31838,76790,India,04-04-2020,"APNA is taking small initiatives,which are to ...",Extremely Positive


Для каждого твитта указано:


*   UserName - имя пользователя, заменено на целое число для анонимности
*   ScreenName - отображающееся имя пользователя, заменено на целое число для анонимности
*   Location - местоположение
*   TweetAt - дата создания твитта
*   OriginalTweet - текст твитта
*   Sentiment - эмоциональная окраска твитта (целевая переменная)



## Задание 1 Подготовка (0.5 балла)

Целевая переменная находится в колонке `Sentiment`.  Преобразуйте ее таким образом, чтобы она стала бинарной: 1 - если у твитта положительная или очень положительная эмоциональная окраска и 0 - если отрицательная или очень отрицательная.

In [3]:
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x in ('Positive', 'Extremely Positive') else 0)
df.sample(4)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
26822,36685,81637,Worldwide,07-04-2020,Los Angeles County asks not to go to the super...,0
1054,5076,50028,"Selangor , MY ?",17-03-2020,Keep yourself safe at home during this covid-1...,1
22533,31338,76290,NaN,03-04-2020,Share prices in the Securities Exchange were i...,1
975,4980,49932,NaN,17-03-2020,If you are one of the people that has an essen...,1


Сбалансированы ли классы?

In [4]:
print(df['Sentiment'].value_counts())

Sentiment
1    18046
0    15398
Name: count, dtype: int64


**Ответ:** Классы сбалансированы достаточно, но не идеально

Выведете на экран информацию о пропусках в данных. Если пропуски присутствуют заполните их строкой 'Unknown'.

In [5]:
print("Missing data in columns before fill:")
for i in df.head():
    print(i, df[i].isna().sum())
for i in df.head():
    df[i].fillna("Unknown", inplace=True)
print("Missing data in columns after fill:")
for i in df.head():
    print(i, df[i].isna().sum())

Missing data in columns before fill:
UserName 0
ScreenName 0
Location 7049
TweetAt 0
OriginalTweet 0
Sentiment 0
Missing data in columns after fill:
UserName 0
ScreenName 0
Location 0
TweetAt 0
OriginalTweet 0
Sentiment 0


C:\Users\Dmitriy\AppData\Local\Temp\ipykernel_12104\2872222514.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[i].fillna("Unknown", inplace=True)


Разделите данные на обучающие и тестовые в соотношении 7 : 3 и укажите `random_state=0`

In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.3, stratify=df['Sentiment'], random_state=0)

## Задание 2 Токенизация (3 балла)

Постройте словарь на основе обучающей выборки и посчитайте количество встреч каждого токена с использованием самой простой токенизации - деления текстов по пробельным символам и приведения токенов в нижний регистр.

In [13]:
from collections import Counter

words = []
print(df["OriginalTweet"].head(5))

for i in df["OriginalTweet"]:
    for word in i.split():
        words.append(word.lower())

words = Counter(words)
words_count = words.most_common()
words_count_df = pd.DataFrame(words_count, columns=['Word', 'Count'])

words_count_df

0    advice Talk to your neighbours family to excha...
1    Coronavirus Australia: Woolworths to give elde...
2    My food stock is not the only one which is emp...
3    Me, ready to go at supermarket during the #COV...
4    As news of the regionÃÂs first confirmed COV...
Name: OriginalTweet, dtype: object


,Word,Count
0,the,38250
1,to,33447
2,and,20935
3,of,18578
4,a,16667
...,...,...
103195,@tartiicat,1
103196,new/used,1
103197,rift,1
103198,$700.00,1


Какой размер словаря получился?

**Ответ:** Как мы видим, получилось 103200 строк, значит столько же различных токенов

Выведите 10 самых популярных токенов с количеством встреч каждого из них. Объясните, почему именно эти токены в топе.

In [14]:
words_count_df.head(10)

,Word,Count
0,the,38250
1,to,33447
2,and,20935
3,of,18578
4,a,16667
5,in,16024
6,for,12193
7,#coronavirus,11759
8,is,10596
9,are,9958


**Ответ:** Логично, что артикли, предлоги и ```#coronavirus``` оказались в топе, потому что артикли и предлоги - сами по себе частые связки в тексте, а хештег - просто тематика всех текстов, логично, что такой хештег будет 

Удалите стоп-слова из словаря и выведите новый топ-10 токенов (и количество встреч) по популярности.  Что можно сказать  о нем?

In [33]:
import nltk
from nltk.corpus import stopwords

# nltk.download('stopwords')
stops = set(stopwords.words('english'))
words_count_df_copy = words_count_df[~words_count_df["Word"].isin(stops)]
words_count_df_copy.head(10)

,Word,Count
7,#coronavirus,11759
16,prices,5625
18,food,5409
23,grocery,4882
24,supermarket,4662
25,covid-19,4504
26,people,4488
27,store,4486
35,#covid19,3561
39,consumer,3233


**Ответ:** Видим, что теперь токены очистились от мусора - артиклей, предлогов и местоимений, которые, логично, есть в каждом тексте, и токены отражают обычные хештеги или самые частые слова. Я вижу, что многие из них связаны с едой или пищевой промышленностью. Возможно, это произошло, потому что еда - главное в выживании во времена коронавируса, а также супермаркеты - неизбежное скопление людей.

Также выведите 20 самых непопулярных слов (если самых непопулярных слов больше, выведите любые 20 из них) Почему эти токены непопулярны, требуется ли как-то дополнительно работать с ними?

In [34]:
words_count_df_copy.tail(20)

,Word,Count
103180,bullion's,1
103181,#safe-haven,1
103182,"$1,715.25/ounce",1
103183,dec.,1
103184,"$1,722.20",1
103185,"rs.46,215",1
103186,gms.,1
103187,https://t.co/s8coy5vvgn,1
103188,home??,1
103189,https://t.co/v8xdxhqeyn,1


**Ответ:** Эти токены непопулярны, потому что содержат пунктуацию, ссылки, конкретные числа/цифры или непопулярные хештеги. Я считаю, что их нужно обработать, удалив пунктуацию

Теперь воспользуемся токенайзером получше - TweetTokenizer из библиотеки nltk. Примените его и посмотрите на топ-10 популярных слов. Чем он отличается от топа, который получался раньше? Почему?

In [35]:
from nltk.tokenize import TweetTokenizer

tokenizer = TweetTokenizer()

words = []

for i in df["OriginalTweet"]:
    words += tokenizer.tokenize(i)

words = Counter(words)
words_count = words.most_common()
words_count_df = pd.DataFrame(words_count, columns=['Word', 'Count'])

words_count_df

,Word,Count
0,the,34781
1,.,34284
2,to,32812
3,",",25142
4,and,20439
...,...,...
87328,martinsville,1
87329,@TartiiCat,1
87330,Rift,1
87331,700.00,1


**Ответ:** На удивление, сейчас токенайзер выдал даже более худшие результаты, чем "рабоче-крестьянский" метод с split, потому что наш новый токенайзер отделяет знаки пунктуации от основного текста, и, естественно, знаков пунктуации в топе много.

Удалите из словаря стоп-слова и пунктуацию, посмотрите на новый топ-10 слов с количеством встреч, есть ли теперь в нем что-то не похожее на слова?

In [39]:
from string import punctuation

punctuation_list = list(punctuation)
words_count_df_copy = words_count_df[
    ~words_count_df["Word"].isin(punctuation_list) & ~words_count_df["Word"].isin(stops)]
words_count_df_copy

,Word,Count
10,Â,10498
11,,10361
13,#coronavirus,10211
14,19,10142
18,I,7484
...,...,...
87328,martinsville,1
87329,@TartiiCat,1
87330,Rift,1
87331,700.00,1


**Ответ:** Да, после очистки от мусора можно видеть а-ля адекватный список топа слов. Однако все еще присутствует мусор - нечитаемые символы, возможно, служебные, возможно, смайлики.

Скорее всего в некоторых топах были неотображаемые символы или отдельные буквы не латинского алфавита. Уберем их: удалите из словаря токены из одного символа, позиция которого в таблице Unicode 128 и более (`ord(x) >= 128`)

Выведите топ-10 самых популярных и топ-20 непопулярных слов. Чем полученные топы отличаются от итоговых топов, полученных при использовании токенизации по пробелам? Что теперь лучше, а что хуже?

In [44]:
words_count_df_copy = words_count_df_copy[
    (words_count_df_copy["Word"].str.len() > 1) |
    (words_count_df_copy["Word"].apply(
        lambda x: ord(x) if len(x) == 1 else 0
    ) <= 127)
    ]

In [45]:
words_count_df_copy.head(10)

,Word,Count
13,#coronavirus,10211
14,19,10142
18,I,7484
23,prices,6166
24,COVID,5945
26,food,5423
29,store,5234
33,supermarket,4803
35,grocery,4350
36,people,4300


In [46]:
words_count_df_copy.tail(20)

,Word,Count
87313,https://t.co/qUQ8Y0uM6n,1
87314,@MajangChien,1
87315,bullion's,1
87316,#safe-haven,1
87317,"1,715",1
87318,"1,722",1
87319,"46,215",1
87320,gms,1
87321,https://t.co/S8coY5VVgN,1
87322,https://t.co/v8XDXhqeYN,1


**Ответ:** Теперь топ похож на реальный, немного отличается от нашего прошлого способа (в том числе, потому что мы не приводили слова к нижнему регистру). Из плюсов: разделения стали лучше, потому что не учитывается пунктуация и маленькие различия между словами, из минусов: появились местоимения.

Выведите топ-10 популярных хештегов (токены, первые символы которых - #) с количеством встреч. Что можно сказать о них?

In [47]:
words_count_df_copy[words_count_df["Word"].str[0] == '#'].head(10)

C:\Users\Dmitriy\AppData\Local\Temp\ipykernel_12104\533891431.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  words_count_df_copy[words_count_df["Word"].str[0] == '#'].head(10)


,Word,Count
13,#coronavirus,10211
60,#COVID19,2621
75,#Covid_19,2126
90,#Coronavirus,1806
116,#COVID2019,1341
159,#toiletpaper,944
175,#covid19,829
186,#COVID,775
246,#CoronaCrisis,599
284,#CoronaVirus,525


**Ответ:** О хештегах можно сказать следующее: все хештеги, очевидно, упоминают эпидемию. Один забавный хештег смог затесаться в этот топ - #toiletpaper, все мы знаем, почему :))

То же самое проделайте для ссылок на сайт https://t.co Сравнима ли популярность ссылок с популярностью хештегов? Будет ли информация о ссылке на конкретную страницу полезна?

In [50]:
print(words_count_df_copy[words_count_df["Word"].str.startswith('https://t.co')].head(10))

                          Word  Count
11014  https://t.co/oXA7SWtoNd      6
12562  https://t.co/G63RP042HO      5
13125  https://t.co/R7sAGojsjg      4
13354  https://t.co/WrLHYzIzAA      4
13709  https://t.co/ymsEmlVTTd      4
13993  https://t.co/3kFUIOJXEp      4
14604  https://t.co/OI39zSAnQ8      4
14916  https://t.co/6yVYKIAb2c      4
14917  https://t.co/xPcm2Xkj4O      4
14951  https://t.co/gu6B4XpqP4      4


C:\Users\Dmitriy\AppData\Local\Temp\ipykernel_12104\2305966469.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(words_count_df_copy[words_count_df["Word"].str.startswith('https://t.co')].head(10))


**Ответ:** Так как топ весьма маленький (мало повторений) и по ссылкам не всегда понятна окраска сообщения, в котором была вставлена ссылка, то информация, которую она несёт, в том виде, в котором мы видим сейчас, невелика.

Используем опыт предыдущих экспериментов и напишем собственный токенайзер, улучшив TweetTokenizer. Функция tokenize должна:



*   Привести текст в нижний регистр
*   Применить TweetTokenizer для  выделения токенов
*   Удалить стоп-слова, пунктуацию, токены из одного символа с позицией в таблице Unicode 128 и более,  ссылки на t.co



In [54]:
def custom_tokenizer(text):
    tokenizer = TweetTokenizer()

    words = tokenizer.tokenize(text.lower())

    words = Counter(words)
    words_count = words.most_common()
    words_count_df = pd.DataFrame(words_count, columns=['Word', 'Count'])

    words_count_df_copy = words_count_df[~words_count_df["Word"].isin(punctuation_list) & ~words_count_df["Word"].isin(stops)]

    words_count_df_copy = words_count_df_copy[
        (words_count_df_copy["Word"].str.len() > 1) |
        (words_count_df_copy["Word"].apply(
            lambda x: ord(x) if len(x) == 1 else 0
        ) <= 127)
        ]
    
    words_count_df_copy = words_count_df_copy[~words_count_df["Word"].str.startswith('https://t.co')]
    
    return words_count_df_copy


In [55]:
custom_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext')

C:\Users\Dmitriy\AppData\Local\Temp\ipykernel_12104\697399648.py:19: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  words_count_df_copy = words_count_df_copy[~words_count_df["Word"].str.startswith('https://t.co')]


,Word,Count
4,sample,1
5,text,1
6,@sample_text,1
10,#sampletext,1


## Задание 3 Векторизация текстов (2 балла)

Обучите CountVectorizer с использованием custom_tokenizer в качестве токенайзера. Как размер полученного словаря соотносится с размером изначального словаря из начала задания 2?

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer  # -- YOUR CODE HERE --

print(len(cv.vocabulary_))

45308


**Ответ:** # -- YOUR ANSWER HERE --

Посмотрим на какой-нибудь конкретный твитт:

In [ ]:
ind = 9023
train.iloc[ind]['OriginalTweet'], train.iloc[ind]['Sentiment']

Автор твитта не доволен ситуацией с едой во Франции и текст имеет резко негативную окраску.

Примените обученный CountVectorizer для векторизации данного текста, и попытайтесь определить самый важный токен и самый неважный токен (токен, компонента которого в векторе максимальна/минимальна, без учета 0). Хорошо ли они определились, почему?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Теперь примените TfidfVectorizer и  определите самый важный/неважный токены. Хорошо ли определились, почему?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Найдите какой-нибудь положительно окрашенный твитт, где TfidfVectorizer хорошо (полезно для определения окраски) выделяет важный токен, поясните пример.

*Подсказка:* явно положительные твитты можно искать при помощи положительных слов (good, great, amazing и т. д.)

In [ ]:
train[train['OriginalTweet'].apply(lambda x: 'your_good_word_here' in x) & (train['Sentiment'] == 1)]

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 Обучение первых моделей (1 балл)

Примените оба векторайзера для получения матриц с признаками текстов.  Выделите целевую переменную.

In [ ]:
# -- YOUR CODE HERE --

Обучите логистическую регрессию на векторах из обоих векторайзеров. Посчитайте долю правильных ответов на обучающих и тестовых данных. Какой векторайзер показал лучший результат? Что можно сказать о моделях?

Используйте `sparse` матрицы (после векторизации), не превращайте их в `numpy.ndarray` или `pd.DataFrame` - может не хватить памяти.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 5 Стемминг (0.5 балла)

Для уменьшения словаря можно использовать стемминг.

Модифицируйте написанный токенайзер, добавив в него стемминг с использованием SnowballStemmer. Обучите Count- и Tfidf- векторайзеры. Как изменился размер словаря?

In [ ]:
def custom_stem_tokenizer(text):
    # -- YOUR CODE HERE --
    return tokens

In [ ]:
custom_stem_tokenizer(
    'This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext adding more words to check stemming')

['sampl', 'text', '@sample_text', '#sampletext', 'ad', 'word', 'check', 'stem']

In [ ]:
cv = CountVectorizer  # -- YOUR CODE HERE --

print(len(cv.vocabulary_))

36652


**Ответ** # -- YOUR ANSWER HERE --

Обучите логистическую регрессию с использованием обоих векторайзеров. Изменилось ли качество? Есть ли смысл применять стемминг?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание  6 Работа с частотами (1.5 балла)

Еще один способ уменьшить количество признаков - это использовать параметры min_df и max_df при построении векторайзера  эти параметры помогают ограничить требуемую частоту встречаемости токена в документах.

По умолчанию берутся все токены, которые встретились хотя бы один раз.



Подберите max_df такой, что размер словаря будет 36651 (на 1 меньше, чем было). Почему параметр получился такой большой/маленький?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        max_df=  # -- YOUR CODE HERE --
                        ).fit(
    # -- YOUR CODE HERE --
)
print(len(cv_df.vocabulary_))

36651


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Подберите min_df (используйте дефолтное значение max_df) в CountVectorizer таким образом, чтобы размер словаря был 3700 токенов (при использовании токенайзера со стеммингом), а качество осталось таким же, как и было. Что можно сказать о результатах?

In [ ]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=  # -- YOUR CODE HERE --
                        ).fit(
    # -- YOUR CODE HERE --
)
print(len(cv_df.vocabulary_))

3700


In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

В предыдущих заданиях признаки не скалировались. Отскалируйте данные (при словаре размера 3.7 тысяч, векторизованные CountVectorizer), обучите логистическую регрессию, посмотрите качество и выведите `barplot`, содержащий по 10 токенов, с наибольшим по модулю положительными/отрицательными весами. Что можно сказать об этих токенах?

In [ ]:
from sklearn.preprocessing import StandardScaler
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 7 Другие признаки (1.5 балла)

Мы были сконцентрированы на работе с текстами твиттов и не использовали другие признаки - имена пользователя, дату и местоположение

Изучите признаки UserName и ScreenName. полезны ли они? Если полезны, то закодируйте их, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Изучите признак TweetAt в обучающей выборке: преобразуйте его к типу datetime и нарисуйте его гистограмму с разделением по цвету на основе целевой переменной. Полезен ли он? Если полезен, то закодируйте его, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --



Поработайте с признаком Location в обучающей выборке. Сколько уникальных значений?

In [ ]:
# -- YOUR CODE HERE --

Постройте гистограмму топ-10 по популярности местоположений (исключая Unknown)

In [ ]:
# -- YOUR CODE HERE --

Видно, что многие местоположения включают в себя более точное название места, чем другие (Например, у некоторых стоит London, UK; а у некоторых просто UK или United Kingdom).

Создайте новый признак WiderLocation, который содержит самое широкое местоположение (например, из London, UK должно получиться UK). Сколько уникальных категорий теперь? Постройте аналогичную гистограмму.

In [ ]:
# -- YOUR CODE HERE --

Закодируйте признак WiderLocation с помощью OHE таким образом, чтобы создались только столбцы для местоположений, которые встречаются более одного раза. Сколько таких значений?


In [ ]:
# -- YOUR CODE HERE --

Добавьте этот признак к матрице отскалированных текстовых признаков, обучите логистическую регрессию, замерьте качество. Как оно изменилось? Оказался ли признак полезным?


*Подсказка:* используйте параметр `categories` в энкодере.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --